In [1]:
import findspark
import pyspark
from pyspark import SparkContext,SparkConf
import numpy as np

In [2]:
findspark.init('C:\spark-3.2.0-bin-hadoop3.2')

In [3]:
findspark.find()

'C:\\spark-3.2.0-bin-hadoop3.2'

In [4]:
sc = pyspark.SparkConf()

In [5]:
sc = SparkContext.getOrCreate() 

In [6]:
a = sc.parallelize([1,2,3,4])

In [7]:
a.collect()

[1, 2, 3, 4]

In [8]:
type(a)

pyspark.rdd.RDD

In [9]:
sc.stop()

# First way to create spark context

In [10]:
conf = SparkConf().setAppName('ishti').setMaster('local')
sc = SparkContext(conf=conf)

In [11]:
sc.getConf().getAll()

[('spark.master', 'local'),
 ('spark.app.id', 'local-1635254841677'),
 ('spark.driver.port', '50263'),
 ('spark.app.name', 'ishti'),
 ('spark.rdd.compress', 'True'),
 ('spark.driver.host', 'ADMIN11-PC'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.submit.pyFiles', ''),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.app.startTime', '1635254841227')]

In [12]:
sc.stop()

# 2nd way 

In [13]:
sc = SparkContext()

In [14]:
sc.getConf().getAll()

[('spark.driver.port', '50294'),
 ('spark.driver.host', 'ADMIN11-PC'),
 ('spark.rdd.compress', 'True'),
 ('spark.app.startTime', '1635254842319'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.id', 'local-1635254842706'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.app.name', 'pyspark-shell')]

In [15]:
sc.getConf()

In [16]:
sc.stop()

In [17]:
sc = SparkContext('local','myapp')

In [18]:
sc.getConf().getAll()

[('spark.master', 'local'),
 ('spark.app.name', 'myapp'),
 ('spark.driver.host', 'ADMIN11-PC'),
 ('spark.rdd.compress', 'True'),
 ('spark.app.id', 'local-1635254843533'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.submit.pyFiles', ''),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.startTime', '1635254843343'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.port', '50325')]

# Actions and Transformations

In machine learning we interact with data by appliying certain commands like groupby sorting min max cleaning graphing etc. All there are termed as operations. In spark we perform operations via actions and transformations. 
difference is transformations mainly deal with allotment of data to Resilient distributed dataset (RDD) ie it focuses on the creation of rdd's and does not execute further until an action is performed hence it is termed as lazy operator. While actions result in an output which is fast as compared to conventional working transformations so not result in any kind of output rather they they just allot data to rdd and do not go further

In [19]:
names = sc.parallelize(['abc','efg','hij','klm','nop','qrs','tuv','wxyz'],3)

In [20]:
names.collect()

['abc', 'efg', 'hij', 'klm', 'nop', 'qrs', 'tuv', 'wxyz']

in general we dont use collect in reality. this is because of 2 reasons:
1. spark is mainly used when size of our dataset is usually bigger than the ram size so if we collect data entire data would be thrown to ram which dissolves the purpose of using spark
2. m still working on this part...

so what do we use to view data here...???
like we used head in pandas we use take and we specify the number of values we want to view. try 
names.take(1)[0][1]

In [21]:
names.take(5)

['abc', 'efg', 'hij', 'klm', 'nop']

In [22]:
type(names)

pyspark.rdd.RDD

# Actions

In [23]:
names.countByValue()

defaultdict(int,
            {'abc': 1,
             'efg': 1,
             'hij': 1,
             'klm': 1,
             'nop': 1,
             'qrs': 1,
             'tuv': 1,
             'wxyz': 1})

Glom function
1. Creates partition in rdd
2. glom helps in accessing elements by means of partition. when we want to view data we generally use take(num)/ take(num)[index] but with glom it becomes more accesible. Check codes below
3. its a pipeline rdd
4. pyspark does not allow worker to refer to other portions so by using glom worker can access other partitions.. still need to understand this 

In [24]:
no = sc.parallelize([3,2,6,7,2,5,8,3,9,3,6,4,7])

In [25]:
no.collect()

[3, 2, 6, 7, 2, 5, 8, 3, 9, 3, 6, 4, 7]

In [26]:
no.glom().collect()

[[3, 2, 6, 7, 2, 5, 8, 3, 9, 3, 6, 4, 7]]

In [27]:
no.take(3)[2]              #if indes is >= take(num) we will get an error
                           #which means we can only see value from that range of take

6

In [28]:
type(no)

pyspark.rdd.RDD

it looks same till this point, now lets try a partition in no rdd and check

In [29]:
no = sc.parallelize([3,2,6,7,2,5,8,3,9,3,6,4,7],2)

In [30]:
no.collect()

[3, 2, 6, 7, 2, 5, 8, 3, 9, 3, 6, 4, 7]

In [31]:
no.glom().collect()

[[3, 2, 6, 7, 2, 5], [8, 3, 9, 3, 6, 4, 7]]

numslices in argument refers to the number of partition we want in our rdd

In [32]:
no.glom().collect()[1][4:]

[6, 4, 7]

In [33]:
type(no.glom())

pyspark.rdd.PipelinedRDD

# Reduce and Fold

In [34]:
nos = sc.parallelize([2,3,4])

In [35]:
nos.reduce(lambda x,y: x+y)

9

In [36]:
def mul(a,b):
    return a*b

In [37]:
nos.reduce(mul)

24

In [38]:
nos.reduce(lambda a,b:a*2+b*2)

28

In [39]:
nos.reduce(lambda x,y: x if x>y else y)    #checking for greatest integer in rdd

4

In [40]:
nos.reduce(lambda x,y: x if x<y else y)

2

Reduce basically performs same as pandas reduce. it works on the principle of comparison between two variables or by means of two var. obviously since we want a reduce operation which isnt possible by working with one variable. so + * > < and such functions would work well with reduce

Now lets see some fold operations

In [41]:
nos = sc.parallelize([2,3,4])  #default partition value is chosen one

In [42]:
nos.collect()

[2, 3, 4]

In [43]:
nos.glom().collect()

[[2, 3, 4]]

In [44]:
nos.fold(0,lambda a,b:a+b)

9

In [45]:
nos = sc.parallelize([2,3,4],2)

In [46]:
nos.collect()

[2, 3, 4]

In [47]:
nos.glom().collect()

[[2], [3, 4]]

In [48]:
nos.fold(1,lambda a,b:a+b)

12

In [49]:
nos.fold(2, lambda a,b:a+b)

15

nos.fold(arg,func) so arg value is being multiplied with the nos of brackets as per patition
1. arg=1 and part=2 ie 3 brackets so 1*3 would get added to total sum
2. arg=2 and part=2 ie 3 brackets so 2*3=6 would be added to total sum

In [50]:
nos = sc.parallelize([2,3,4],3)

In [51]:
nos.glom().collect()

[[2], [3], [4]]

In [52]:
nos.fold(3,lambda x,y:x+y)

21

lets understand this by doing some other operations

In [53]:
nos = sc.parallelize([2,3,4])

In [54]:
nos.glom().collect()

[[2, 3, 4]]

In [55]:
nos.fold(0,lambda x,y:x*y)

0

In [56]:
nos.fold(1,lambda x,y:x*y)

24

In [57]:
nos.fold(2,lambda x,y:x*y)  #2*3*4=24 and 2fold *2brackets=4 so 24*4=96

96

(arg^^(no of brackets))*total is what fold will return in multiplication case

In [58]:
nos = sc.parallelize([2,3,4],3)

In [59]:
nos.glom().collect()

[[2], [3], [4]]

In [60]:
nos.fold(3,lambda x,y:x*y)

1944

3^4= 81
total = 24
o/p = 81*24--> 1944

Other ways of applying fold.

we can use multiply division and other operations as well from operator class

In [61]:
numbers = sc.parallelize(np.random.rand(10))

In [62]:
numbers.collect()

[0.38148337351830364,
 0.6382965714292532,
 0.9069796756995182,
 0.1118109078483076,
 0.3591637251081974,
 0.01442548007477118,
 0.06618036527451532,
 0.2381547988025724,
 0.9144281796359204,
 0.9974606054117435]

In [63]:
from operator import add

In [64]:
numbers.fold(1,add)

6.6283836828031015

Note: All functions applied above belong to action category

Further on we ll be seeing transformation operations

# Transformations

We have two types of transformations namely:
1. Narrow
2. Wide

# Narrow transformations

In [65]:
num = sc.parallelize(range(1,6))

In [66]:
num.collect()  #collect is action

[1, 2, 3, 4, 5]

In [67]:
num.map(lambda x:x*2)      

PythonRDD[38] at RDD at PythonRDD.scala:53

In [68]:
num.map(lambda x:x*2).collect()

[2, 4, 6, 8, 10]

 1. map is a transformation function here
 2. we can see that it transformed one rdd to another rdd
 3. lambda function wont get executed until we apply an action method on top of transformation
 4. hence the name lazy operation

In [69]:
def into(x):
    return x*3

In [70]:
num.map(into).collect()

[3, 6, 9, 12, 15]

In [71]:
num.map(into)

PythonRDD[41] at RDD at PythonRDD.scala:53

In [72]:
names = sc.parallelize(['dom','som','rom','dooo'])

In [73]:
names.map(lambda x: 'Mr.'+x).collect()

['Mr.dom', 'Mr.som', 'Mr.rom', 'Mr.dooo']

In [74]:
rdd = sc.parallelize([2,3,4])

In [75]:
rdd.flatMap(lambda x: range(2,x)).collect()

[2, 2, 3]

In [76]:
rdd.flatMap(lambda y:range(1,y)).collect()

[1, 1, 2, 1, 2, 3]

In [77]:
rdd.flatMap(lambda x:(x,x*2,x*3)).collect()

[2, 4, 6, 3, 6, 9, 4, 8, 12]

In [78]:
rdd.flatMap(lambda x:(x,x-1)).glom().collect()

[[2, 1, 3, 2, 4, 3]]

In [79]:
rdd.filter(lambda x:x%2==0).collect()

[2, 4]

In [80]:
names.take(4)

['dom', 'som', 'rom', 'dooo']

In [81]:
names.filter(lambda x: 'd' in x).collect()

['dom', 'dooo']

map flatmap filter are all transformation functions 
1. Map keeps the number of partions same as original partition but changes some features of rdd like it wont be similar to the original rdd
2. Flatmap can increase or decrease the number of partitions in an rdd in short it performs operations such that ur new rdd will be different from original one interms of size features
3. filter will only reduce the number of partitions in an rdd

Logic in each function of transformation defines the way we give input to a particular function 
for eg we cant give a single variable function to flatmap as it wont serve the logic and we cant give a double variable function to filter 

refer the codes to understand it better 

In [82]:
num.collect()

[1, 2, 3, 4, 5]

In [83]:
nos.collect()

[2, 3, 4]

In [84]:
num.union(nos)

UnionRDD[53] at union at NativeMethodAccessorImpl.java:0

In [85]:
num.union(nos).collect()

[1, 2, 3, 4, 5, 2, 3, 4]

In [86]:
nos.union(num).collect()

[2, 3, 4, 1, 2, 3, 4, 5]

In [87]:
nos.sample(True,.4).collect()

[2, 2, 3, 3, 3, 3, 4]

# Wide transformations

In [88]:
names.collect()

['dom', 'som', 'rom', 'dooo']

In [89]:
names_gr = names.groupBy(lambda x:x[0])

In [90]:
names_gr.collect()

[('d', <pyspark.resultiterable.ResultIterable at 0x255422bc5e0>),
 ('s', <pyspark.resultiterable.ResultIterable at 0x255422fcb80>),
 ('r', <pyspark.resultiterable.ResultIterable at 0x255422fc460>)]

what groupby does...?
as per the function mentioned above groupby is taking the first element of each variable and naming the grp with that initial. now inside that grp we have those specific variables which have their first letter with that grp name. So basically we have 
1. grp name as first letter ie: key
2. variables as values to those keys

So in short we have a key value pair formed. now we ll access that using a for loop

Note: This logic is also termed as generator

In [91]:
for i,j in names_gr.collect():
    print(i,list(j))

d ['dom', 'dooo']
s ['som']
r ['rom']


In [92]:
aa = sc.parallelize([2,1,3,8,6,4])

In [93]:
bb = sc.parallelize([5,2,4])

In [94]:
aa_grp3 = aa.groupBy(lambda x:x%3).collect()

In [95]:
for i,j in aa_grp:
    print(i,list(j))

NameError: name 'aa_grp' is not defined

In [ ]:
aa_grp2 = aa.groupBy(lambda x:x%2).collect()

In [ ]:
for i,j in aa_grp2:
    print(i,list(j))

How is the above code working..?

1. values which are divisible are put under 0 key category ie remainder 0
2. those which arent and have a remainder are put under 1:key category

imply the same logic when we divide by 3

In [ ]:
aa.intersection(bb).collect()

In [ ]:
bb.intersection(aa).collect()

In [ ]:
aa.collect()

In [ ]:
bb.collect()

In [ ]:
aa.subtract(bb).collect()

In [ ]:
bb.subtract(aa).collect()

In [ ]:
aa.distinct().collect()

In [ ]:
cc = sc.parallelize([2,2,3,3,4,4])

In [ ]:
cc.distinct().collect()

Union Intersection Subtract Distinct
1. intersection and union is basically like mathematics sets union and intersection
2. Subtract works different
if we have aa.subtract(bb) then it looks for all elements of aa in bb and whichever are not common it prints that value. Like intersection of aa and bb but will print remaining elements of aa apart from intersection elements
3. distinct gives distinct values

25th OCT: Till now we were using single value in rdd now we will try working with key value pairs in rdd and perform some functions of wide transformations

In [ ]:
data = sc.parallelize([(1,2),(1,3),(2,3),(3,4),(3,5),(1,2),(2,3),(3,4)])

In [ ]:
data.countByValue()

In [ ]:
data.collect()

In [ ]:
data.count()

In [ ]:
data.keys().collect()

In [ ]:
data.values().collect()

In [ ]:
data.lookup(3)

In [ ]:
data.mapValues(lambda x:x*x).collect()

In [ ]:
data.reduceByKey(lambda x,y: x+y).collect()

In [ ]:
data.reduceByKey(max).collect()   # output contains those tuples which have the max value

In [ ]:
data.reduceByKey(min).collect()    # output contains those tuples which have the min value

In [ ]:
data_grp = data.groupBy(lambda x:x[0])

In [ ]:
data_grp.collect()

In [ ]:
for i,j in data_grp.collect():
    print(i,list(j))

In [ ]:
data_grp1 = data.groupByKey().collect()

In [ ]:
data_grp1

In [ ]:
for i,j in data_grp1:
    print(i,list(j))

Can you spot the difference between groupby and groupbykey code above...?

When would we use groupbykey or a reducebykey or mapvalue instead of groupby/reduce/map/flatmap...?
1. when we have a key value pair in our rdd
2. in order to access a key or a value we would specify that via a function and access that respectively like mapvalue means map over values of rdd

In [ ]:
dat = sc.parallelize([(1,4),(3,2)])
dat.flatMapValues(lambda x:range(1,x)).collect()
# similar to flatmap as seen previously

In [ ]:
data2 = sc.parallelize([(2,5),(1,4),(5,1)])
data2.collect()

In [ ]:
data.collect()

In [ ]:
data.subtractByKey(data2).collect()   
# keys of data2 are matched with data and the remaining pairs from data are given as o/p

In [ ]:
data2.subtractByKey(data).collect()

In [ ]:
data.join(data2).collect()    # not clear how this worked...!!!

In [ ]:
data2.join(data).collect()

In [ ]:
data.rightOuterJoin(data2).collect()     # extended code of join, understand join first!

In [ ]:
data2.rightOuterJoin(data).collect()

In [ ]:
data.leftOuterJoin(data2).collect()

In [ ]:
sc.stop()